In [ ]:
!pip install pytorch_lightning
import torch
import torchvision
import torchmetrics
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader
import pytorch_lightning as pl

     |████████████████████████████████| 582 kB 3.9 MB/s 
     |████████████████████████████████| 596 kB 27.5 MB/s 
     |████████████████████████████████| 398 kB 40.5 MB/s 
     |████████████████████████████████| 136 kB 40.0 MB/s 
     |████████████████████████████████| 1.1 MB 39.1 MB/s 
     |████████████████████████████████| 144 kB 52.6 MB/s 
     |████████████████████████████████| 271 kB 40.8 MB/s 
     |████████████████████████████████| 94 kB 3.3 MB/s 
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Uninstalling typing-extensions-3.10.0.2:
      Successfully uninstalled typing-extensions-3.10.0.2
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow

In [ ]:
class LightningMNISTClassifier(pl.LightningModule):

    def __init__(self):
        super().__init__()
        
        self.conv1 = nn.Conv2d(1,6,5, padding = 2)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(6,16,9)
        self.fc1 = nn.Linear(3*3*16,64)
        self.fc2 = nn.Linear(64,10)

        self.valid_acc = torchmetrics.Accuracy()
        self.test_acc = torchmetrics.Accuracy()

    def forward(self, x):
        x = self.pool(F.leaky_relu(self.conv1(x)))
        x = self.pool(F.leaky_relu(self.conv2(x)))
        x = torch.flatten(x, 1) 
        x = F.leaky_relu(self.fc1(x))
        x = self.fc2(x)

        return x

    def cross_entropy_loss(self, logits, labels):
        return F.cross_entropy(logits, labels, )

    def training_step(self, train_batch, batch_idx):
        x, y = train_batch
        logits = self.forward(x)
        loss = self.cross_entropy_loss(logits, y)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, val_batch, batch_idx):
        x, y = val_batch
        logits = self.forward(x)
        loss = self.cross_entropy_loss(logits, y)
        self.log('val_loss', loss)
        self.valid_acc(logits, y)
        self.log('valid_acc', self.valid_acc)

    def test_step(self, test_batch, batch_idx):
        x, y = test_batch
        logits = self.forward(x)
        loss = self.cross_entropy_loss(logits, y)
        self.log('test_loss', loss)
        self.test_acc(logits, y)
        self.log('test_acc', self.test_acc)

    def configure_optimizers(self):
        #optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-4)
        return optimizer

In [ ]:
class MNISTDataModule(pl.LightningDataModule):

    def setup(self, stage):
        transform = transforms.Compose(
            [transforms.ToTensor(),
             transforms.Normalize(mean=[0.5],
                                 std=[0.5])])
      
        # prepare transforms standard to MNIST
        self.trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform)
        self.testset = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform)
    def train_dataloader(self):
        return DataLoader(self.trainset, batch_size=32,
                                          shuffle=True, num_workers=2)

    def val_dataloader(self):
        return DataLoader(self.testset, batch_size=32,
                                         shuffle=False, num_workers=2)


In [ ]:
AVAIL_GPUS = min(1, torch.cuda.device_count())

data_module = MNISTDataModule()

model = LightningMNISTClassifier()
trainer = pl.Trainer(gpus = AVAIL_GPUS, max_epochs = 10)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(model, data_module)

Missing logger folder: /content/lightning_logs


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [ ]:
torch.save(model.state_dict(), 'cnn_mnist.pth')

#torch.save(model, './models/mnist.pt')
#torch.load('./models/mnist.pt')

In [ ]:
state_dict = torch.load('checkpoint.pth')
model.load_state_dict(state_dict)
#trainer = pl.Trainer(limit_train_batches=0, limit_val_batches=0)
#trainer.fit(model, data_module)

<All keys matched successfully>

In [ ]:
class SVHNDataModule(pl.LightningDataModule):

    def setup(self, stage):
        transform = transforms.Compose(
            [transforms.Grayscale(1),
             transforms.Resize(28),
             transforms.ToTensor(),
             transforms.Normalize(mean=[0.5],
                                 std=[0.5])])
      
        # prepare transforms standard to MNIST
        self.trainset = torchvision.datasets.SVHN(root='./data', split='train',#train=True,
                                        download=True, transform=transform)
        self.testset = torchvision.datasets.SVHN(root='./data', split='test',#train=False,
                                       download=True, transform=transform)
    def train_dataloader(self):
        return DataLoader(self.trainset, batch_size=32,
                                          shuffle=True, num_workers=2)
        
    def val_dataloader(self):
        return DataLoader(self.testset, batch_size=32,
                                         shuffle=False, num_workers=2)

    def test_dataloader(self):
        return DataLoader(self.testset, batch_size=32,
                                         shuffle=False, num_workers=2)

In [ ]:
svhn_data_module = SVHNDataModule()
trainer.test(model, svhn_data_module)

  0%|          | 0/182040794 [00:00<?, ?it/s]

  0%|          | 0/64275384 [00:00<?, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.2596803903579712
        test_loss           2.5482845306396484
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_acc': 0.2596803903579712, 'test_loss': 2.5482845306396484}]

In [ ]:
trainer = pl.Trainer(gpus = AVAIL_GPUS, max_epochs = 10)
trainer.fit(model, svhn_data_module)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Using downloaded and verified file: ./data/train_32x32.mat
Using downloaded and verified file: ./data/test_32x32.mat


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type      | Params
----------------------------------------
0 | conv1     | Conv2d    | 156   
1 | pool      | MaxPool2d | 0     
2 | conv2     | Conv2d    | 7.8 K 
3 | fc1       | Linear    | 9.3 K 
4 | fc2       | Linear    | 650   
5 | valid_acc | Accuracy  | 0     
6 | test_acc  | Accuracy  | 0     
----------------------------------------
17.9 K    Trainable params
0         Non-trainable params
17.9 K    Total params
0.072     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [ ]:
trainer.test(model, svhn_data_module)
torch.save(model.state_dict(), 'cnn_transfer_svhn.pth')

Using downloaded and verified file: ./data/train_32x32.mat
Using downloaded and verified file: ./data/test_32x32.mat


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.8349339365959167
        test_loss           0.6064464449882507
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
